In [2]:
import seaborn as sns
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
 df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [7]:
df.time


0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [9]:
df['time']=encoder.fit_transform(df['time'])

In [10]:
df.time

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [11]:
## Independent and Dependent feature
X=df.drop(labels=['time'],axis=1)
y=df.time

In [12]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train ,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [13]:
## Pipeline 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## handle missing value 
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## Categorical to numerical
from sklearn.compose import ColumnTransformer

In [28]:
categorical_cols = ['sex','smoker','day']
numericals_cols = ['total_bill','tip','size']

In [29]:
## Feature Engineering Automation
## Numerical Pipelines
num_pipeline = Pipeline(
   steps=[
       ('imputer',SimpleImputer(strategy='median')),## Missing Values
       ('scaler',StandardScaler()), ##Feature scaling
   ]
)

#categorical pipelines
cat_pipeline = Pipeline(
               steps=[
                   ('imputer',SimpleImputer(strategy='most_frequent')), ## Handle missing value
                   ('onehotencoder',OneHotEncoder()) ## Categorical to numerical features
               ]
)

In [30]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numericals_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols),
])

In [66]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

ValueError: Specifying the columns using strings is only supported for dataframes.

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [80]:
## Model training Automation
models={
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

In [81]:
from sklearn.metrics import accuracy_score,classification_report

In [82]:
def evalute_model(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model=list(models.values())[i]
        ##Train Model
        model.fit(X_train,y_train)
        
        
        #Predict Testing Data
        y_test_pred=model.predict(X_test)
        
        #Get Accuracy scores for Train and Test data
        
        test_model_score = accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
        return report
    
    

In [83]:
evalute_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877}

In [84]:
classifier= RandomForestClassifier()

In [85]:
##HyperParameter Tuning
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [86]:
from sklearn.model_selection import RandomizedSearchCV

In [88]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [89]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.897 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.949 total time=   0.0s
[CV 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [90]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'entropy'}